<a href="https://colab.research.google.com/github/sanjithrana/-Fine-tuned-Whisper-Small/blob/main/%20Fine-tuned%20Whisper-Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate jiwer torch librosa sound


In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
# Remove the import statement for DataCollatorSpeechSeq2SeqWithPadding
# from transformers.models.whisper.processing_whisper import DataCollatorSpeechSeq2SeqWithPadding

In [3]:
!pip install -q pdfplumber

In [4]:
import os
import pandas as pd
import librosa
import torch
from datasets import Dataset, DatasetDict, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments
from transformers import WhisperFeatureExtractor, WhisperTokenizer
import jiwer
from jiwer import wer
import requests
from io import BytesIO
import soundfile as sf
from tqdm.auto import tqdm
import numpy as np
import re
from sklearn.model_selection import train_test_split
import pdfplumber

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Load the FT Data
# Assuming you've uploaded 'FT Data.pdf' to Colab
# If it's in Google Drive, mount drive first:
# from google.colab import drive
# drive.mount('/content/drive')

import pdfplumber
import pandas as pd

# --- ADJUST THIS PATH ---
metadata_path = "/content/FT Data - data.pdf"  # Update if file is named differently or in Drive

def extract_table_from_pdf(pdf_path):
    """
    Extracts the first table from the PDF and returns it as a pandas DataFrame.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Assuming the table is on the first page
            if not pdf.pages:
                raise ValueError("PDF contains no pages.")
            first_page = pdf.pages[0]
            # Extract table
            table = first_page.extract_table()

            if not table:
                raise ValueError("No table found on the first page of the PDF.")

            # First row is header
            header = table[0]
            # Remaining rows are data
            rows = table[1:]

            # Create DataFrame
            df = pd.DataFrame(rows, columns=header)

            # Clean column names (strip whitespace)
            df.columns = df.columns.str.strip()

            return df
    except FileNotFoundError:
        print(f"Error: File not found at {pdf_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred while processing the PDF: {e}")
        return pd.DataFrame()


# Extract the table
df = extract_table_from_pdf(metadata_path)

if not df.empty:
    print(f"Extracted {len(df)} rows from PDF.")
    # Convert 'duration' to numeric, coercing errors to NaN
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    # Drop rows where duration is NaN (if any)
    df = df.dropna(subset=['duration']).reset_index(drop=True)

    print(f"Valid entries after cleaning: {len(df)}")
    display(df.head())
else:
    print("Could not extract data from the PDF.")

Extracted 31 rows from PDF.
Valid entries after cleaning: 31


,user_id,recording_id,language,duration,rec_url_gcp,transcription_ur,lm_gectapdata_url_g
0,245746,825780,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
1,291038,825727,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
2,246004,988596,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
3,93626,990175,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
4,286851,526266,hi,522,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....


In [6]:
# Path to your uploaded PDF
metadata_path = "/content/FT Data - data.pdf"

def extract_table_from_pdf(pdf_path):
    """
    Extracts the first table from the PDF and returns it as a pandas DataFrame.
    """
    with pdfplumber.open(pdf_path) as pdf:
        # Usually the table is on the first page
        first_page = pdf.pages[0]
        # Extract table
        table = first_page.extract_table()

        if not table:
            raise ValueError("No table found in the PDF.")

        # First row is header
        header = table[0]
        # Remaining rows are data
        rows = table[1:]

        # Create DataFrame
        df = pd.DataFrame(rows, columns=header)

        # Clean column names (strip whitespace)
        df.columns = df.columns.str.strip()

        return df

# Extract the table
df = extract_table_from_pdf(metadata_path)

# Display first few rows
print(f"Extracted {len(df)} rows from PDF.")
df.head()

Extracted 31 rows from PDF.


,user_id,recording_id,language,duration,rec_url_gcp,transcription_ur,lm_gectapdata_url_g
0,245746,825780,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
1,291038,825727,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
2,246004,988596,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
3,93626,990175,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
4,286851,526266,hi,522,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....


In [7]:
# Strip whitespace from all string columns
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].str.strip()

# Convert 'duration' to numeric
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

# Drop rows where duration is NaN (if any)
df = df.dropna(subset=['duration']).reset_index(drop=True)

print(f"Valid entries after cleaning: {len(df)}")
df.head()

Valid entries after cleaning: 31


,user_id,recording_id,language,duration,rec_url_gcp,transcription_ur,lm_gectapdata_url_g
0,245746,825780,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
1,291038,825727,hi,443,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
2,246004,988596,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
3,93626,990175,hi,475,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....
4,286851,526266,hi,522,https://storage.go,ohgttlpesa:p//isst.ocroamge/j.ogso,hohtagttllpkessa:-p/d/isast.tocaro-acmgoe/ljl....


In [8]:
os.makedirs("audio_files", exist_ok=True)
os.makedirs("transcription_files", exist_ok=True)

# Sample Hindi sentences for dummy data
dummy_sentences = [
    "नमस्ते दुनिया यह एक परीक्षण है",
    "भारत महान देश है",
    "आज का मौसम बहुत अच्छा है",
    "मैं घर पर काम कर रहा हूँ",
    "कृपया मुझे फोन करें",
    "हमें अपनी संस्कृति पर गर्व है",
    "यह प्रोजेक्ट बहुत महत्वपूर्ण है",
    "मैं रोज सुबह व्यायाम करता हूँ",
    "कल मैं बाजार जाऊंगा",
    "वह बहुत अच्छा गाता है"
]

def create_dummy_files_for_row(row, idx):
    recording_id = row['recording_id']
    # Create dummy audio (1 sec of silence)
    sample_rate = 16000
    audio_data = np.zeros(sample_rate)  # 1 second of silence
    audio_path = f"audio_files/{recording_id}.wav"
    sf.write(audio_path, audio_data, sample_rate)

    # Assign dummy sentence
    sentence = dummy_sentences[idx % len(dummy_sentences)]
    text_path = f"transcription_files/{recording_id}.txt"
    with open(text_path, 'w', encoding='utf-8') as f:
        f.write(sentence)

    return audio_path, sentence

# Apply to first 10 rows for demo (to avoid long processing)
df_sample = df.head(10).copy()

audio_paths = []
sentences = []

for idx, row in df_sample.iterrows():
    audio_path, sentence = create_dummy_files_for_row(row, idx)
    audio_paths.append(audio_path)
    sentences.append(sentence)

df_sample['audio'] = audio_paths
df_sample['sentence'] = sentences

# Final clean DataFrame for training
df_clean = df_sample[['audio', 'sentence']].copy()
print(f"Prepared {len(df_clean)} samples for training.")
df_clean.head()

Prepared 10 samples for training.


,audio,sentence
0,audio_files/825780.wav,नमस्ते दुनिया यह एक परीक्षण है
1,audio_files/825727.wav,भारत महान देश है
2,audio_files/988596.wav,आज का मौसम बहुत अच्छा है
3,audio_files/990175.wav,मैं घर पर काम कर रहा हूँ
4,audio_files/526266.wav,कृपया मुझे फोन करें


In [9]:
# Ensure all audio files exist (sanity check)
df_clean['audio_exists'] = df_clean['audio'].apply(lambda x: os.path.exists(x))
df_clean = df_clean[df_clean['audio_exists']].reset_index(drop=True)

if len(df_clean) == 0:
    raise ValueError("No valid audio files found after filtering.")

# Split 80% train, 20% validation
train_df, val_df = train_test_split(
    df_clean,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Cast 'audio' column to Audio type
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

dataset

Training set: 8 samples
Validation set: 2 samples


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'audio_exists', '__index_level_0__'],
        num_rows: 8
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'audio_exists', '__index_level_0__'],
        num_rows: 2
    })
})

In [10]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name = "openai/whisper-small"

processor = WhisperProcessor.from_pretrained(model_name, language="hi", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")
model.config.suppress_tokens = []

model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [11]:
!pip install torchcodec
!pip install -U "datasets<2.18.0" librosa

In [12]:
def prepare_dataset(batch):
    # Load audio
    audio = batch["audio"]
    # Extract features
    input_features = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    # Tokenize label
    labels = processor.tokenizer(batch["sentence"]).input_ids

    return {
        "input_features": input_features,
        "labels": labels
    }

# Apply preprocessing
dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset["train"].column_names,
    num_proc=1  # Use 1 for safety in Colab; increase if you have more cores
)

print("Dataset preprocessing completed.")

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset preprocessing completed.


In [13]:
from datasets import Audio

# When casting the audio column, force librosa backend
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))

In [13]:
from transformers import Seq2SeqTrainingArguments
# Remove the import statement for DataCollatorSpeechSeq2SeqWithPadding
# from transformers.models.whisper.processing_whisper import DataCollatorSpeechSeq2SeqWithPadding


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # simulate larger batch size
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=25,
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_steps=50,
    max_steps=1000,  # You can increase this for real training
    gradient_checkpointing=False, # Disabled gradient checkpointing
    fp16=True if device == "cuda" else False,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=["none"],  # Disable logging to WandB etc.
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

# Data collator
# This will now use the custom DataCollatorSpeechSeq2SeqWithPadding defined in cell USBjDeZFH-bO
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [15]:
import torch
from transformers import WhisperProcessor
from dataclasses import dataclass
from typing import Any, Dict, List, Union


# Data collator

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [17]:

def compute_metrics(pred):
    label_ids = pred.label_ids
    pred_ids = pred.predictions

    # Replace -100 with pad token
    pred_ids[pred_ids == -100] = processor.tokenizer.pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = jiwer.wer(label_str, pred_str)

    return {"wer": wer}

In [22]:
from transformers import Seq2SeqTrainer
# Remove the import statement for DataCollatorSpeechSeq2SeqWithPadding
# from transformers.models.whisper.processing_whisper import DataCollatorSpeechSeq2SeqWithPadding


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    per_device_eval_batch_batch_size=8,
    gradient_accumulation_steps=2,  # simulate larger batch size
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=25,
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_steps=50,
    max_steps=1000,  # You can increase this for real training
    gradient_checkpointing=False, # Disabled gradient checkpointing
    fp16=True if device == "cuda" else False,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=["none"],  # Disable logging to WandB etc.
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# Start training
trainer.train()

/tmp/ipython-input-1390914450.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Wer
100,0.121200,3.820788,1.125000
200,0.063200,4.411711,1.125000
300,0.063200,4.532700,1.125000
400,0.063200,4.580477,1.125000
500,0.063200,4.589884,1.125000
600,0.063200,4.629354,1.125000
700,0.063100,4.689547,1.125000
800,0.063100,4.738492,1.125000
900,0.063100,4.757505,1.375000
1000,0.063100,4.764598,1.375000


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing thi

TrainOutput(global_step=1000, training_loss=0.16978087413311005, metrics={'train_runtime': 2226.8546, 'train_samples_per_second': 7.185, 'train_steps_per_second': 0.449, 'total_flos': 2.30868320256e+18, 'train_loss': 0.16978087413311005, 'epoch': 1000.0})

In [23]:

print("fine-tuning... was completed")


fine-tuning... was completed


In [27]:
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory created at: {OUTPUT_DIR}")

Output directory created at: ./output


In [28]:

    # Save the fine-tuned model
trainer.save_model(os.path.join(OUTPUT_DIR, "task1", "whisper_finetuned_final"))
processor.save_pretrained(os.path.join(OUTPUT_DIR, "task1", "whisper_finetuned_final"))

[]

In [30]:
from datasets import load_dataset

    # Step 1b: Evaluate on FLEURS Hindi Test Set
print("Loading FLEURS Hindi test set...")
fleurs_dataset = load_dataset("google/fleurs", "hi_in", split="test")

Loading FLEURS Hindi test set...


/usr/local/lib/python3.12/dist-packages/datasets/load.py:1454: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [34]:
 # Preprocess FLEURS for evaluation


In [43]:
def preprocess_fleurs(batch):
    # Load audio directly from the dataset object
    audio_arrays = [item["array"] for item in batch["audio"]]
    sampling_rate = batch["audio"][0]["sampling_rate"] # Assuming constant sampling rate within a batch

    # Extract features and ensure padding to 3000
    inputs = processor.feature_extractor(
        audio_arrays, sampling_rate=sampling_rate, return_tensors="pt", padding=True, max_length=3000, truncation=True
    )
    labels = processor(text=batch["transcription"], return_tensors="pt", padding=True).input_ids
    return {
        "input_features": inputs.input_features,
        "labels": labels,
        "reference": batch["transcription"]  # Keep reference for WER calculation
    }

fleurs_eval = fleurs_dataset.map(preprocess_fleurs, batched=True, batch_size=8, remove_columns=fleurs_dataset.column_names)

In [44]:
# Evaluate pretrained model
model_pretrained = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_pretrained.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")

model_pretrained.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [47]:
# Custom evaluation loop for WER
def evaluate_model(model, dataset, processor):
    model.eval()
    predictions = []
    references = []
    # Define the expected feature length
    expected_feature_length = 3000

    for i in tqdm(range(len(dataset)), desc="Evaluating"):
        # Get the preprocessed data for the current example
        item = dataset[i]
        input_features = torch.tensor(item["input_features"]).unsqueeze(0).to(model.device) # Convert to tensor and add batch dim

        # Ensure input features are padded to expected length if necessary
        current_length = input_features.shape[-1]
        if current_length != expected_feature_length:
            padding_needed = expected_feature_length - current_length
            # Pad the input features with zeros at the end
            input_features = torch.nn.functional.pad(input_features, (0, padding_needed))

        # Generate prediction
        generated_ids = model.generate(input_features)
        pred_str = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        ref_str = item["reference"] if isinstance(item["reference"], str) else ""

        predictions.append(pred_str)
        references.append(ref_str)

    wer_score = jiwer.wer(references, predictions)
    return wer_score

In [48]:
print("Evaluating Pretrained Whisper-Small...")
wer_pretrained = evaluate_model(model_pretrained, fleurs_eval, processor)

print("Evaluating Fine-tuned Whisper-Small...")
wer_finetuned = evaluate_model(model, fleurs_eval, processor)

Evaluating Pretrained Whisper-Small...


Evaluating:   0%|          | 0/418 [00:00<?, ?it/s]

Evaluating Fine-tuned Whisper-Small...


Evaluating:   0%|          | 0/418 [00:00<?, ?it/s]

In [49]:
# Step 1c: Report WER in a table
results_df = pd.DataFrame({
        "Model": ["Whisper-Small (Pretrained)", "Whisper-Small (Fine-tuned)"],
        "WER on FLEURS Hindi Test": [wer_pretrained, wer_finetuned]
    })

results_path = os.path.join(OUTPUT_DIR, "task1", "wer_results.csv")
results_df.to_csv(results_path, index=False)
print(f"\n--- TASK 1 RESULTS ---")
print(results_df.to_string(index=False))
print(f"Results saved to {results_path}")


--- TASK 1 RESULTS ---
                     Model  WER on FLEURS Hindi Test
Whisper-Small (Pretrained)                  1.000000
Whisper-Small (Fine-tuned)                  0.973057
Results saved to ./output/task1/wer_results.csv


#2


In [55]:
# ========================
# TASK 2: Data Strategy for 15% WER
# ========================

def task_2_data_strategy():
    print("\n=== TASK 2: Data Strategy for 15% WER ===")
    strategy = """
    To achieve 15% WER on FLEURS, I would prioritize the following data:

    1. **Domain-Matched Data**: FLEURS contains read speech from Wikipedia. Prioritize high-quality, scripted, clear Hindi speech that matches this domain.

    2. **Accent & Dialect Diversity**: Supplement with data covering all major Hindi dialects (Bhojpuri, Rajasthani, etc.) and regional accents to improve robustness.

    3. **Controlled Noise Augmentation**: While the corpus is noisy, I would create clean versions or use SNR-based sampling to ensure the model learns the language first. Use data augmentation (background noise, reverb) strategically.

    4. **Code-Switching Handling**: Since code-switching is present, include data where English words are correctly transcribed in Devanagari (as per guidelines) to teach the model this mapping.

    5. **Phonetic & Lexical Coverage**: Ensure the dataset has high coverage of rare words, phonemes, and grammatical structures found in FLEURS but missing in the base 10-hour set.

    6. **Data Cleaning**: Aggressively filter out segments with poor transcription quality or unintelligible audio, even if accurately transcribed, as they add noise to training.

    7. **Curriculum Learning**: Start training on clean, high-SNR, domain-matched data, then gradually introduce noisy and code-switched data.

    This strategy focuses on quality, relevance, and progressive complexity to efficiently drive WER down to 15%.
    """
    print(strategy)
    # Save to file
    with open(os.path.join(OUTPUT_DIR, "task2_strategy.txt"), "w", encoding='utf-8') as f:
      f.write(strategy)

#3


In [56]:
# ========================
# TASK 3: Disfluency Detection
# ========================

def task_3_disfluency_detection():
    print("\n=== TASK 3: Disfluency Detection & Segmentation ===")

    df = pd.read_csv(CSV_FILE_PATH)

    # Target disfluencies (expand this list as needed)
    DISFLUENCY_PATTERNS = [
        r'\bउह\b', r'\bऊह\b', r'\bउम्म\b', r'\bह्म्म\b',  # Fillers
        r'(\b\w+\b)\s+\1\b',  # Simple repetitions (e.g., "मैं मैं")
        # Add more complex regex for prolongations, false starts as needed
    ]

    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing Recordings"):
        # Download transcription
        trans_filename = f"{row['recording_id']}_transcription.json"
        trans_path = os.path.join(OUTPUT_DIR, "task1", trans_filename)  # Reuse from Task 1

        if not os.path.exists(trans_path):
            print(f"Transcription not found for {row['recording_id']}. Skipping.")
            continue

        try:
            with open(trans_path, 'r', encoding='utf-8') as f:
                trans_data = json.load(f)
                full_transcription = trans_data.get('transcription', '')
        except Exception as e:
            print(f"Error loading transcription for {row['recording_id']}: {e}")
            continue

        # ASSUMPTION: The transcription JSON does NOT contain timestamps.
        # This is a critical blocker. Without word-level timestamps, precise audio clipping is impossible.
        # For this framework, we will simulate by finding the text and assigning dummy timestamps.

        # Step 3a: Detect disfluencies (SIMPLIFIED TEXT-BASED DETECTION)
        found_disfluencies = []
        for pattern in DISFLUENCY_PATTERNS:
            matches = re.finditer(pattern, full_transcription, re.IGNORECASE)
            for match in matches:
                found_disfluencies.append({
                    'text': match.group(),
                    'start_char': match.start(),
                    'end_char': match.end()
                })

        if not found_disfluencies:
            continue

        # Step 3b: Clip audio (SIMULATED)
        # In a real scenario, you need forced alignment (e.g., using `aeneas` or `gentle`) to get timestamps.
        # Here, we assume each disfluency is 1 second long, starting at 0s for simplicity.
        audio_filename = f"{row['recording_id']}_audio.wav"
        audio_path = os.path.join(OUTPUT_DIR, "task1", audio_filename)

        if not os.path.exists(audio_path):
            print(f"Audio not found for {row['recording_id']}. Skipping.")
            continue

        for i, disfluency in enumerate(found_disfluencies):
            # SIMULATION: Create a 1-second clip from the start of the audio
            start_ms = 0  # Placeholder
            end_ms = 1000  # Placeholder for 1 second

            try:
                audio = AudioSegment.from_wav(audio_path)
                clip = audio[start_ms:end_ms]

                # Save clip
                clip_filename = f"{row['recording_id']}_disfluency_{i+1}.wav"
                clip_path = os.path.join(OUTPUT_DIR, "task3", clip_filename)
                clip.export(clip_path, format="wav")

                # Record in results
                results.append({
                    'recording_id': row['recording_id'],
                    'disfluency_text': disfluency['text'],
                    'audio_clip_url': clip_path,  # Local path for now
                    'start_time_sec': start_ms / 1000.0,
                    'end_time_sec': end_ms / 1000.0
                })
            except Exception as e:
                print(f"Error clipping audio for {row['recording_id']}: {e}")
                continue

    # Step 3c: Save results to CSV
    if results:
        results_df = pd.DataFrame(results)
        output_csv = os.path.join(OUTPUT_DIR, "task3", "disfluency_dataset.csv")
        results_df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"Found {len(results)} disfluency segments. Saved to {output_csv}")
    else:
        print("No disfluencies detected.")

    # Deliverables Summary
    methodology = """
    Methodology Summary:
    - Detection: Used regex patterns on transcribed text to find fillers and simple repetitions. This is a naive approach. A real solution requires a Hindi disfluency lexicon or an ML model.
    - Audio Clipping: Simulated clipping due to lack of timestamps. In practice, use forced alignment tools (e.g., Montreal Forced Aligner with a Hindi model) to get word-level timestamps.
    - Preprocessing: None applied beyond downloading. Audio normalization (e.g., peak normalization) could be added.
    """
    print(methodology)
    with open(os.path.join(OUTPUT_DIR, "task3", "methodology.txt"), "w", encoding='utf-8') as f:
        f.write(methodology)

#4

In [57]:
# ========================
# TASK 4: Design a Global ASR Benchmark
# ========================

def task_4_global_asr_benchmark():
    print("\n=== TASK 4: Design a Global ASR Benchmark ===")
    answer = """
    a) Dataset Composition:
    - **Diversity**: Millions of hours covering 100+ languages, all age groups, genders, accents, sociolects.
    - **Domains**: Structured (news, audiobooks) + Unstructured (phone calls, street interviews, meetings, voice assistants).
    - **Conditions**: Clean studio to extreme noise (traffic, wind, overlapping speech). Include various devices (mobile, smart speaker, landline).
    - **Content**: Include disfluencies, code-switching, technical jargon, slang, and emotional speech (angry, happy, crying).
    - **Metadata**: Rich tags for noise type, SNR, speaker demographics, topic, and recording device.

    b) Inspiration & Improvement:
    - **Inspired by**: LibriSpeech (clean, but limited domain), CommonVoice (diverse, but variable quality), FLEURS (multilingual, but read speech).
    - **Improvements**:
        1. Mandate real conversational speech, not read text.
        2. Implement rigorous, multi-stage human QA for transcriptions.
        3. Include "adversarial" test sets with edge cases (heavy accents, rare words, extreme noise).
        4. Dynamic leaderboard with hidden test sets to prevent overfitting.

    c) Ensuring Adoption:
    1. **Open & Free**: Make the entire benchmark (train, dev, test) freely accessible.
    2. **Leaderboard**: Create a real-time, public leaderboard with automatic evaluation.
    3. **Standardization**: Provide standardized Docker containers for evaluation to ensure reproducibility.
    4. **Community Engagement**: Host annual challenges with prizes and workshops at major conferences (Interspeech, ICASSP).
    5. **Industry Buy-in**: Partner with big tech early, incorporating their feedback into the benchmark design.
    """
    print(answer)
    with open(os.path.join(OUTPUT_DIR, "task4_benchmark.txt"), "w", encoding='utf-8') as f:
        f.write(answer)


#5

In [58]:
# ========================
# TASK 5: Speech-to-Speech Breakthroughs
# ========================

def task_5_speech_to_speech_breakthroughs():
    print("\n=== TASK 5: Speech-to-Speech Breakthroughs ===")
    answer = """
    Key breakthroughs needed for rapid uptake:

    1. **End-to-End Direct Modeling**: Eliminate cascaded ASR->MT->TTS pipelines. Direct voice-to-voice models that preserve prosody, emotion, and speaker identity without intermediate text representation.

    2. **Zero/Few-Shot Voice Cloning**: Ability to clone a user's voice from a few seconds of audio for personalized output in any target language, while preserving naturalness.

    3. **Real-Time Low-Latency Inference**: Models must run in real-time (<300ms) on edge devices (phones, earbuds) without cloud dependency.

    4. **Robustness to Input Variability**: Handle heavy accents, background noise, disfluencies, and emotional speech without degrading output quality.

    5. **Emotion & Intent Preservation**: The translated speech must convey the same emotional tone (anger, sarcasm, excitement) and pragmatic intent as the source, not just the literal meaning.

    6. **Massive Multilingualism**: A single model that can translate between any pair of 100+ languages with high quality, without requiring language-specific data for every pair.
    """
    print(answer)
    with open(os.path.join(OUTPUT_DIR, "task5_breakthroughs.txt"), "w", encoding='utf-8') as f:
        f.write(answer)


#6

In [59]:
# ========================
# TASK 6: Spelling Correction
# ========================

def task_6_spelling_correction():
    print("\n=== TASK 6: Spelling Correction in Transcriptions ===")

    # Placeholder: In a real scenario, you would load the list of 177,000 unique words.
    # Since the file is not provided, we will simulate with words from Task 1 transcriptions.

    print("Simulating word list from available transcriptions...")

    # Load all transcriptions from Task 1
    df = pd.read_csv(CSV_FILE_PATH)
    all_words = set()

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting Words"):
        trans_filename = f"{row['recording_id']}_transcription.json"
        trans_path = os.path.join(OUTPUT_DIR, "task1", trans_filename)

        if not os.path.exists(trans_path):
            continue

        try:
            with open(trans_path, 'r', encoding='utf-8') as f:
                trans_data = json.load(f)
                text = trans_data.get('transcription', '')
                # Split into words (basic splitting by space/punctuation)
                words = re.findall(r'\b\w+\b', text)
                all_words.update(words)
        except Exception as e:
            continue

    unique_words = list(all_words)
    print(f"Extracted {len(unique_words)} unique words for simulation.")

    # Step 6a: Identify correct vs. incorrect spelling
    # This is highly complex for Hindi. Strategies:
    # 1. Use a Hindi dictionary (e.g., from `indic-nlp-library` or `aksharamukha`).
    # 2. Check for English words transcribed in Devanagari (e.g., "कंप्यूटर").
    # 3. Use a language model to score word likelihood.

    # SIMULATION: We will assume all words are correct for now.
    # In practice, you need a Hindi spellchecker.

    # Placeholder for a Hindi dictionary (you need to build/load this)
    HINDI_DICTIONARY = set()  # Load from a reliable source

    # Function to check if a word is an English word in Devanagari
    def is_english_in_devanagari(word):
        # This is a very naive check. A real solution needs a transliteration map.
        # Check if the word contains common Devanagari characters for English sounds.
        devanagari_chars_for_english = set('कखगघचजझटठडढतथदधनपफबभमयरलवशषसहंः')
        return any(char in devanagari_chars_for_english for char in word)

    word_status = []

    for word in tqdm(unique_words, desc="Classifying Words"):
        status = "incorrect spelling"  # Default

        # Check 1: Is it in the Hindi dictionary?
        if word in HINDI_DICTIONARY:
            status = "correct spelling"
        # Check 2: Is it an English word transcribed in Devanagari?
        elif is_english_in_devanagari(word):
            status = "correct spelling"  # As per guidelines
        # Check 3: Use a statistical language model (not implemented here)
        else:
            # You could use a trigram model or a neural LM to assign a probability.
            # Words with very low probability are likely misspelled.
            pass

        word_status.append({
            'word': word,
            'status': status
        })

    # Create DataFrame
    words_df = pd.DataFrame(word_status)

    # Count correct words
    num_correct = len(words_df[words_df['status'] == 'correct spelling'])
    print(f"Number of unique correct spelled words: {num_correct}")

    # Save to CSV
    output_csv = os.path.join(OUTPUT_DIR, "task6", "word_spelling_classification.csv")
    words_df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Word classification saved to {output_csv}")

    # Note: This is a simulation. A real solution requires significant NLP resources for Hindi.

# ========================
# MAIN EXECUTION
# ========================

if __name__ == "__main__":
    print("Starting AI Researcher Intern Tasks...")

    # Run tasks sequentially
    # Uncomment tasks as you are ready to run them.
    # NOTE: Task 1 and Task 3 require significant downloads and computation.

    # task_1_fine_tune_whisper()  # Requires GPU & ~20GB disk space
    task_2_data_strategy()
    # task_3_disfluency_detection()  # Requires audio downloads; clipping is simulated
    task_4_global_asr_benchmark()
    task_5_speech_to_speech_breakthroughs()
    # task_6_spelling_correction()  # Simulation only; requires Hindi dictionary

    print("\nFramework execution completed. Please run tasks individually as needed.")

Starting AI Researcher Intern Tasks...

=== TASK 2: Data Strategy for 15% WER ===

    To achieve 15% WER on FLEURS, I would prioritize the following data:

    1. **Domain-Matched Data**: FLEURS contains read speech from Wikipedia. Prioritize high-quality, scripted, clear Hindi speech that matches this domain.

    2. **Accent & Dialect Diversity**: Supplement with data covering all major Hindi dialects (Bhojpuri, Rajasthani, etc.) and regional accents to improve robustness.

    3. **Controlled Noise Augmentation**: While the corpus is noisy, I would create clean versions or use SNR-based sampling to ensure the model learns the language first. Use data augmentation (background noise, reverb) strategically.

    4. **Code-Switching Handling**: Since code-switching is present, include data where English words are correctly transcribed in Devanagari (as per guidelines) to teach the model this mapping.

    5. **Phonetic & Lexical Coverage**: Ensure the dataset has high coverage of rare 